In [1]:
import pandas as pd

In [ ]:
songs = pd.read_csv('songs.csv')

In [ ]:
songs

## Clean the data

In [ ]:
songs.dtypes

In [ ]:
# Why is Pandas treating 'Month' as 'object' data type by default instead of a numeric data type? We are expecting only numeric values in that column.

pd.to_numeric(songs['Month'])

In [ ]:
# There's a 'Jan' value in that column causing errors.

songs['Month'].value_counts()

In [ ]:
# It appears that some months were recorded with written abbreviations.

month_dict = {
    'Jan': 1,
    'Feb': 2,
    'Apr': 4,
    'May': 5,
    'Jun': 6,
    'Sep': 9,
    'Dec': 12
}

songs['Month'].replace(month_dict, inplace=True)
songs['Month'] = pd.to_numeric(songs['Month'])

In [ ]:
songs['Month'].value_counts()

In [ ]:
# Similarly, for 'Length (Duration)', why is Pandas treating the column as 'object' data type by default when we would expect only numeric values?

pd.to_numeric(songs['Length (Duration)'])

In [ ]:
# The commas are causing the problem.

songs['Length (Duration)'] = pd.to_numeric(songs['Length (Duration)'].str.replace(',', ''))

In [ ]:
songs['Length (Duration)'].describe()

## Remove rows and columns with >= 50% missing values

In [ ]:
orig_col = songs.columns

orig_col

In [ ]:
orig_shape = songs.shape

orig_shape

# (number of rows, number of columns)

In [ ]:
songs.dropna(thresh=orig_shape[1]/2, inplace=True)

# keep only rows with at least 8 non-missing values

# axis=0 is the default, so you don't need to specify it

In [ ]:
songs.shape

In [ ]:
songs.dropna(axis=1, thresh=orig_shape[0]/2, inplace=True)

# keep only columns with at least 8 non-missing values

In [ ]:
songs.shape

In [ ]:
songs

In [ ]:
# index column isn't useful

songs.drop('Index', axis=1, inplace=True)

# reset index after deleting rows

songs.reset_index(drop=True, inplace=True)

In [ ]:
new_col = songs.columns

In [ ]:
set(orig_col) - set(new_col)

## Descriptive statistics

In [ ]:
songs.describe()

In [ ]:
# Minimum year is 92? I don't think music was being recorded back then. This must be a mistake.

songs['Year'].sort_values(ascending=True)[:50]

In [ ]:
songs['Year'][1550:1600]

In [ ]:
# It appears that 1992 was recorded as 92 instead.

songs['Year'].replace(92, 1992, inplace=True)

In [ ]:
songs['Year'].describe()

In [ ]:
songs.describe()

In [ ]:
# This seems much more reasonable now.

# Now, to find outliers. We only want to look at the numeric columns.

for col in songs.describe().columns:
    col_mean = songs[col].mean()
    col_std = songs[col].std()

    print(col, 'outliers')

    for value in songs[col]:
        if abs(value - col_mean) > 3 * col_std:
            print(f'{value}, Z-score: {(value - col_mean) / col_std}')

    print('\r')

## Imputation

In [ ]:
import numpy as np
from sklearn.impute import SimpleImputer

In [ ]:
for col in songs:
    print(f'{col} missing values: {songs[col].isnull().sum()}')

In [ ]:
continuous_columns = ['Beats Per Minute (BPM)', 'Energy', 'Loudness (dB)', 'Liveness', 'Length (Duration)', 'Acousticness', 'Speechiness', 'Popularity']

In [ ]:
continuous_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

In [ ]:
continuous_imputed_df = pd.DataFrame(continuous_imputer.fit_transform(songs[continuous_columns]), columns=continuous_columns)

continuous_imputed_df

In [ ]:
for col in continuous_columns:
    songs[col] = continuous_imputed_df[col]

In [ ]:
categorical_columns = ['Artist', 'Top Genre', 'Year']

In [ ]:
categorical_imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

In [ ]:
categorical_imputed_df = pd.DataFrame(categorical_imputer.fit_transform(songs[categorical_columns]), columns=categorical_columns)

categorical_imputed_df

In [ ]:
for col in categorical_columns:
    songs[col] = categorical_imputed_df[col]

In [ ]:
for col in songs:
    print(f'{col} missing values: {songs[col].isnull().sum()}')

## Datetime processing

In [ ]:
# Year column is currently a float because of the missing values. Now that we've gotten rid of them, we can change the type to int.

songs['Year'] = songs['Year'].astype(int)

In [ ]:
songs['date'] = songs['Year'].astype(str) + '-' + songs['Month'].astype(str) + '-' + '1'

In [ ]:
songs['date']

In [ ]:
songs['date'] = pd.to_datetime(songs['date'])

In [ ]:
songs['date']

In [ ]:
songs['age'] = pd.to_datetime('2022-01-18') - songs['date']

In [ ]:
songs['age']

In [ ]:
age = pd.Series([songs['age'][i].days for i in range(len(songs['age']))])

In [ ]:
songs['age'] = age

In [ ]:
songs.drop(['Year', 'Month', 'date'], axis=1, inplace=True)

In [ ]:
songs

## Dummy variables

In [ ]:
songs = pd.get_dummies(songs, columns=['Top Genre'], drop_first=True)

In [ ]:
songs